### CNN DIMENSIONS ANALYSIS

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
class cnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5)      # 1 image, 32 channels, 5*5 krnel default stride=1
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)
        
    
        x = torch.randn(1,50,50).view(-1,1,50,50)
        self.to_linear = None   #auxillary variable to calculate shape of output of conv+max_pool
        self.convs(x)        
        
        self.fc1 = nn.Linear(self.to_linear,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,2)
        
    def convs(self,x):

        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        print("Layer_1=",x[0].shape[0],x[0].shape[1],x[0].shape[2])
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        print("Layer_2=",x[0].shape[0],x[0].shape[1],x[0].shape[2])
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        print("Layer_3=",x[0].shape[0],x[0].shape[1],x[0].shape[2])
        #x = self.model(x)
        if self.to_linear is None:
            self.to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1,self.to_linear)   #flattening
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x,dim=1)

In [20]:
net = cnet()

Layer_1= 32 23 23
Layer_2= 64 9 9
Layer_3= 128 2 2


In [4]:
print(net)

cnet(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)


In [6]:
print(net.to_linear)

512


#### Formula

Input: n X n
<br>
Padding: p
<br>
Stride: s
<br>
Filter size: f X f
<br>
Output: [(n+2p-f)/s  + 1] X [(n+2p-f)/s  +  1]

#First layer  
* conv2d(output 32 channels, input = 50x50, filter=5x5, stride =1)
<br>
(50-5)/1 + 1  = 46 i.e, 32 channels of size 46x46
* maxpool2d(output 32 channels, input = 46x46 , filter=2x2, stride=2 (by default=kernel_size)
<br>
(46-2)/2 + 1 = 23 i.e, 32 channels of size  23x23
<br>
=> 32x23x23
<br>
and so on.....
